In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler, label_binarize
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_predict
from sklearn.pipeline import Pipeline

In [4]:
dtrain = pd.read_csv("D:\Coding_data/fashion-mnist_train.csv")
dtest = pd.read_csv("D:\Coding_data/fashion-mnist_test.csv")

In [11]:
dvalid = dtrain[:5000]
dtrain = dtrain[5000:]

In [12]:
print("Train shape:\n", dtrain.shape)
print("Validation shape:\n", dvalid.shape)
print("Test shape:\n", dtest.shape)

Train shape:
 (55000, 785)
Validation shape:
 (5000, 785)
Test shape:
 (10000, 785)


In [14]:
predict = 'label'

dtrain_X = dtrain.drop(predict, axis=1)
dtrain_y = dtrain[predict]

dtest_X = dtest.drop(predict, axis=1)
dtest_y = dtest[predict]

X_train, X_valid, y_train, y_valid = train_test_split(dtrain_X, dtrain_y, test_size=0.1, random_state=42, stratify=dtrain['label'])